<a href="https://colab.research.google.com/github/kozyreviva/Data_Science/blob/main/timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.
​
Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.
​
Вам нужно:
​
1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.
​

## Загрузка данных

Подключим библиотеки.

In [ ]:
#библиотеки
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import plotly.express as px

Скачаем датасет и сохраним в переменную.

In [ ]:
#скачивание датасета
try:
  df = pd.read_csv("/datasets/taxi.csv", index_col = [0],parse_dates = [0])
except:
  df = pd.read_csv("taxi.csv", index_col = [0], parse_dates = [0])

Выведем датасет и посмотрем краткую информацию.

In [ ]:
#вывод таблицы
df.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [ ]:
#информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


In [ ]:
#размер таблицы
df.shape

(26496, 1)

**Краткие выводы:**  

Датасет содержит данные по вызовам такси за каждые 10 минут. Пропуски в датасете отсутствуют. Для дальнейшей работы с таблицей необходима сортировка индекса. Также необходима сгруппировать данные по часу, это необходимо для построение прогноза.

## Анализ данных

Отсортируем таблицу и сгруппируем данные по часу.

In [ ]:
#сортировка датафрейма
df.sort_index(inplace=True)
#ресемплирование
df = df.resample('1H').sum()

Проведем небольшой анализ данных.

Т.к данных много для наглядности ресемплируем данные по 6 часов и посмотрим на график

In [ ]:
fig = px.line(df.resample("6H").sum(), title = "Количество вызовов")
fig.show()

Посмотрим на тренд и сезонность. Сохраним их в отдельные столбцы.

In [ ]:
#создание объекта структуры с результатами применения функции
decomposed = seasonal_decompose(df)
#создание столбцов
df["trand"] = decomposed.trend
df["season"] = decomposed.seasonal


Вывод тренда.

In [ ]:
fig = px.line(df["trand"], title = "Тренд")
fig.show()

Вывод сезонности. Возьмем интервал в 2 дня для наглядности.

In [ ]:
fig = px.line(df["2018-03-01":"2018-03-02"]["season"], title = "Сезонность")
fig.show()

**Выводы:**  

Таблица содержит нестационарный временной ряд. Это подтверждается восходящим трендом и соответственно изменением среднего значения.

Сезонность показывает что пик вызовов приходится на полночь, а минимум н 6 утра.

## Обучение моделей

Создадим функцию которая добавляет в датафрейм столбцы с годом, месяцем, днём и днём недели. Также добавим столбцы со сдвигами и скользящим средним.

In [ ]:
#функция добавления столбцов
def make_features(data, lag, rolling_mean_size):
    data["year"] = data.index.year
    data["month"] = data.index.month
    data["day"] = data.index.day
    data["dayofweek"] = data.index.dayofweek

    for i in range(1, lag+1):
        data["lag_{}",format(i)] = data["num_orders"].shift(i)
    
    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()


Применение этой функции необходимо для обучения модели. Она предоставляет больше данных.  

Применим функцию.

In [ ]:
#применение функции
make_features(df, 15, 10)

Удалим строки с пропусками образовавшиеся в ходе сдвига данных.

In [ ]:
#удаление строк
df = df.dropna(axis = 0)

Разделим таблицу на обучающую и тестовую выборки при этом данные не смешиваем.  
Разделим на признаки.

In [ ]:
#разделение на выборки
train, test = train_test_split(df, shuffle = False, test_size = 0.1)

#разделение на признаки
features_train = train.drop("num_orders",axis = 1)
target_train = train["num_orders"]

features_test = test.drop("num_orders",axis = 1)
target_test = test["num_orders"]

Создадим валидатор для временных рядов.

In [ ]:
#создание временного валидатора
ts = TimeSeriesSplit(n_splits = 5, test_size = 3)

Протестируем модели кросс-валидацией.

In [ ]:
%%time
#модель дерева решений
model_tree = DecisionTreeRegressor(random_state = 12345)

params = [{"max_depth": np.arange(1,21,2)}]
gs_tree = GridSearchCV(estimator = model_tree,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_tree.fit(features_train.values, target_train.values)
print(gs_tree.best_params_)
print(np.abs(gs_tree.score(features_train.values, target_train.values)))

{'max_depth': 7}
18.997075513588488
CPU times: user 2.14 s, sys: 5.93 ms, total: 2.15 s
Wall time: 2.15 s


In [ ]:
%%time
#модель случайного леса
model_rf = RandomForestRegressor(random_state = 12345)

params = [{"n_estimators": np.arange(10,51,10),
      "max_depth": np.arange(1,16,2)}]
gs_rf = GridSearchCV(estimator = model_rf,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_rf.fit(features_train.values, target_train.values)
print(gs_rf.best_params_)
print(np.abs(gs_rf.score(features_train.values, target_train.values)))

{'max_depth': 7, 'n_estimators': 20}
17.688201798683018
CPU times: user 2min 18s, sys: 214 ms, total: 2min 18s
Wall time: 2min 18s


In [ ]:
%%time
#линейная регрессия
linear_model = LinearRegression()

scores = cross_val_score(linear_model, features_train.values, target_train.values, scoring="neg_root_mean_squared_error", cv = ts)

print(np.abs(scores.mean()))

62.51569903840725
CPU times: user 42.1 ms, sys: 25.1 ms, total: 67.2 ms
Wall time: 64.3 ms


In [ ]:
%%time
#модель LightGBM
model_LGBM = LGBMRegressor()

params = [{"n_estimators": np.arange(10,51,10),
      "max_depth": np.arange(1,21,2)}]
gs_LGBM = GridSearchCV(estimator = model_LGBM,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_LGBM.fit(features_train.values, target_train.values)
print("LightGBM")
print(gs_LGBM.best_params_)
print(np.abs(gs_LGBM.score(features_train.values, target_train.values)))

LightGBM
{'max_depth': 9, 'n_estimators': 40}
16.333249454436515
CPU times: user 35.4 s, sys: 796 ms, total: 36.2 s
Wall time: 19.8 s


## Проверка моделей

Проверим модели на тестовой выборке.

In [ ]:
#модель дерева решений
print("Дерево решений")
print("RMSE:",np.abs(gs_tree.score(features_test.values, target_test.values)))

Дерево решений
RMSE: 52.793722412653786


In [ ]:
#модель рандомного леса
print("Рандомный лес")
print("RMSE:",np.abs(gs_rf.score(features_test.values, target_test.values)))

Рандомный лес
RMSE: 40.2310401292837


In [ ]:
#модель линейной регрессии
linear_model.fit(features_train.values,target_train.values)
predictions = linear_model.predict(features_test.values)
print("Линейная регрессия")
print("RMSE:",np.sqrt(mean_squared_error(target_test.values, predictions)))

Линейная регрессия
RMSE: 40.49237873541757


In [ ]:
#модель LightGBM
print("LightGBM")
print("RMSE:",np.abs(gs_LGBM.score(features_test.values, target_test.values)))

LightGBM
RMSE: 39.717151346166325


**Вывод:**  
Модель градиентного бустинга показала наилучший результат прогнозирования в работе с временными рядами.